## Imports

In [1]:
!ls

pima-diabetes.csv  sample_data


In [2]:
import numpy as np
import pandas as pd

## Load Data

In [3]:
data = pd.read_csv('pima-diabetes.csv')

In [4]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## To numpy

In [5]:
np.set_printoptions(precision=3, formatter={'float': lambda x: f"{x:.3f}"})

In [6]:
np_data = data.to_numpy()

In [7]:
np_data[:5, :]

array([[6.000, 148.000, 72.000, 35.000, 0.000, 33.600, 0.627, 50.000,
        1.000],
       [1.000, 85.000, 66.000, 29.000, 0.000, 26.600, 0.351, 31.000,
        0.000],
       [8.000, 183.000, 64.000, 0.000, 0.000, 23.300, 0.672, 32.000,
        1.000],
       [1.000, 89.000, 66.000, 23.000, 94.000, 28.100, 0.167, 21.000,
        0.000],
       [0.000, 137.000, 40.000, 35.000, 168.000, 43.100, 2.288, 33.000,
        1.000]])

In [8]:
import random
np.random.shuffle(np_data)

In [9]:
np_data[:5, :]

array([[4.000, 118.000, 70.000, 0.000, 0.000, 44.500, 0.904, 26.000,
        0.000],
       [1.000, 122.000, 90.000, 51.000, 220.000, 49.700, 0.325, 31.000,
        1.000],
       [4.000, 97.000, 60.000, 23.000, 0.000, 28.200, 0.443, 22.000,
        0.000],
       [1.000, 146.000, 56.000, 0.000, 0.000, 29.700, 0.564, 29.000,
        0.000],
       [0.000, 100.000, 88.000, 60.000, 110.000, 46.800, 0.962, 31.000,
        0.000]])

## Split

In [10]:
split_index = int(0.2 * len(np_data[:, 0]))

In [11]:
X_valid = np_data[:split_index, :-1]
y_valid = np_data[:split_index, -1]

X_train = np_data[split_index:, 0:-1]
y_train = np_data[split_index:, -1]

In [12]:
print(len(X_valid))
print(len(y_valid))
print(len(X_train))
print(len(y_train))
print(X_valid[:5, :])
print(y_train[:5])

153
153
615
615
[[4.000 118.000 70.000 0.000 0.000 44.500 0.904 26.000]
 [1.000 122.000 90.000 51.000 220.000 49.700 0.325 31.000]
 [4.000 97.000 60.000 23.000 0.000 28.200 0.443 22.000]
 [1.000 146.000 56.000 0.000 0.000 29.700 0.564 29.000]
 [0.000 100.000 88.000 60.000 110.000 46.800 0.962 31.000]]
[1.000 0.000 0.000 1.000 0.000]


## Normalize

In [13]:
mean = X_train.mean(axis=0)
X_train -= mean
std = X_train.std(axis=0)
X_train /= std

X_valid -= mean
X_valid /= std

In [14]:
print(X_valid[:5, :])
print(y_train[:5])
print(X_train.shape)

[[0.048 -0.059 0.077 -1.292 -0.708 1.639 1.301 -0.603]
 [-0.839 0.066 1.086 1.930 1.242 2.304 -0.450 -0.187]
 [0.048 -0.717 -0.427 0.161 -0.708 -0.448 -0.094 -0.936]
 [-0.839 0.818 -0.629 -1.292 -0.708 -0.256 0.272 -0.353]
 [-1.135 -0.623 0.985 2.498 0.267 1.933 1.476 -0.187]]
[1.000 0.000 0.000 1.000 0.000]
(615, 8)


# Create model

## def CustomDense(keras.Layer):

In [15]:
import keras

class CustomDense(keras.Layer):
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.W = self.add_weight(shape=(input_dim, self.units), initializer="random_normal")
        self.b = self.add_weight(shape=(self.units,), initializer="zeros")

    def call(self, inputs):
        y = keras.ops.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

In [17]:
# there is an error in the book here. They pass the str "relu" but its not callable
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(8, activation='relu'),
    CustomDense(4, activation=keras.ops.relu),
    Dense(1, activation="sigmoid")
])

In [18]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [19]:
history = model.fit(X_train, y_train, epochs=256, batch_size=16, validation_data=(X_valid, y_valid))

Epoch 1/256
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.7140 - loss: 0.6818 - val_accuracy: 0.5686 - val_loss: 0.6867
Epoch 2/256
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6588 - loss: 0.6583 - val_accuracy: 0.5752 - val_loss: 0.6638
Epoch 3/256
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6756 - loss: 0.6187 - val_accuracy: 0.6209 - val_loss: 0.6415
Epoch 4/256
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7290 - loss: 0.5650 - val_accuracy: 0.6405 - val_loss: 0.6231
Epoch 5/256
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7445 - loss: 0.5322 - val_accuracy: 0.6667 - val_loss: 0.6100
Epoch 6/256
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7728 - loss: 0.4910 - val_accuracy: 0.6863 - val_loss: 0.5962
Epoch 7/256
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7680 - loss: 0.4932 - val_accuracy: 0.6993 - val_loss: 0.5874
Epoch 8/256
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7430 - loss: 0.4801 - val_accuracy: 0.6993 - 